In [73]:
import numpy as np
import os
import cv2
import torch
import imageio

In [228]:
# Load data
dist_path = '/root/dataset/shan/panorama_motion/archiviz_rot_only/dists'
dists = []

for fname in os.listdir(dist_path):
    dist = imageio.imread(os.path.join(dist_path, fname))
    dists.append(dist)

/tmp/ipykernel_15385/912078718.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  dist = imageio.imread(os.path.join(dist_path, fname))


In [220]:
dist_0 = dists[0]
print(dist_0)

[[0.80392157 0.80392157 0.80392157 ... 0.63529412 0.63137255 0.63137255]
 [0.8        0.8        0.8        ... 0.63137255 0.62745098 0.62745098]
 [0.79607843 0.79607843 0.79607843 ... 0.62745098 0.62352941 0.62352941]
 ...
 [0.22745098 0.22352941 0.21960784 ... 0.00784314 0.01176471 0.01176471]
 [0.22745098 0.22352941 0.21960784 ... 0.00784314 0.01176471 0.01568627]
 [0.22745098 0.22352941 0.21960784 ... 0.01176471 0.01568627 0.01960784]]


In [221]:
freq_bands = 2.**np.linspace(0., 9, num=10)
print(freq_bands)

[  1.   2.   4.   8.  16.  32.  64. 128. 256. 512.]


In [222]:
scaled_dist_0 = (dist_0 / np.max(dist_0))
print(scaled_dist_0)

scaled_freq_bands = freq_bands / np.max(freq_bands)
scaled_freq_bands = np.insert(scaled_freq_bands, 0, 0)
print(scaled_freq_bands)

[[0.80392157 0.80392157 0.80392157 ... 0.63529412 0.63137255 0.63137255]
 [0.8        0.8        0.8        ... 0.63137255 0.62745098 0.62745098]
 [0.79607843 0.79607843 0.79607843 ... 0.62745098 0.62352941 0.62352941]
 ...
 [0.22745098 0.22352941 0.21960784 ... 0.00784314 0.01176471 0.01176471]
 [0.22745098 0.22352941 0.21960784 ... 0.00784314 0.01176471 0.01568627]
 [0.22745098 0.22352941 0.21960784 ... 0.01176471 0.01568627 0.01960784]]
[0.         0.00195312 0.00390625 0.0078125  0.015625   0.03125
 0.0625     0.125      0.25       0.5        1.        ]


In [223]:
ranges = []
clip_values = []
idx = 0

while True:
    ranges.append((scaled_freq_bands[idx], scaled_freq_bands[idx+1]))
    clip_values.append(idx+1)
    idx += 1
    
    if idx == 9:
        break

print(ranges)
print(clip_values)

[(0.0, 0.001953125), (0.001953125, 0.00390625), (0.00390625, 0.0078125), (0.0078125, 0.015625), (0.015625, 0.03125), (0.03125, 0.0625), (0.0625, 0.125), (0.125, 0.25), (0.25, 0.5)]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [180]:
# clip scaled_dist_map
clip_dist_map = np.zeros_like(scaled_dist_0)

for i, (lower, upper) in enumerate(ranges):
    clip_dist_map[(scaled_dist_0>=lower) & (scaled_dist_0 < upper)] = clip_values[i]


[[9. 9. 9. ... 9. 9. 9.]
 [9. 9. 9. ... 9. 9. 9.]
 [9. 9. 9. ... 9. 9. 9.]
 ...
 [8. 8. 8. ... 4. 4. 4.]
 [8. 8. 8. ... 4. 4. 5.]
 [8. 8. 8. ... 4. 5. 5.]]


In [193]:
print(np.sum(scaled_dist_0 == 0))

29567


In [229]:
dists = np.array(dists) / 255.
print(dists)

[[[0.80392157 0.80392157 0.80392157 ... 0.63529412 0.63137255 0.63137255]
  [0.8        0.8        0.8        ... 0.63137255 0.62745098 0.62745098]
  [0.79607843 0.79607843 0.79607843 ... 0.62745098 0.62352941 0.62352941]
  ...
  [0.22745098 0.22352941 0.21960784 ... 0.00784314 0.01176471 0.01176471]
  [0.22745098 0.22352941 0.21960784 ... 0.00784314 0.01176471 0.01568627]
  [0.22745098 0.22352941 0.21960784 ... 0.01176471 0.01568627 0.01960784]]

 [[0.76470588 0.76470588 0.76862745 ... 0.68627451 0.68627451 0.68627451]
  [0.76078431 0.76078431 0.76470588 ... 0.68235294 0.68235294 0.68235294]
  [0.75686275 0.75686275 0.76078431 ... 0.67843137 0.67843137 0.67843137]
  ...
  [0.03137255 0.02745098 0.02745098 ... 0.00784314 0.00392157 0.00392157]
  [0.03529412 0.03137255 0.03137255 ... 0.00392157 0.00392157 0.00784314]
  [0.03529412 0.03529412 0.03529412 ... 0.         0.00392157 0.00784314]]

 [[0.7372549  0.7372549  0.7372549  ... 0.7372549  0.7372549  0.74117647]
  [0.73333333 0.733333

In [9]:
import numpy as np
freq_bands = 2.**np.linspace(0., 9, num=10)
scaled_freq_bands = freq_bands / np.max(freq_bands)
scaled_freq_bands = np.insert(scaled_freq_bands, 0, 0)

print(scaled_freq_bands)

[0.         0.00195312 0.00390625 0.0078125  0.015625   0.03125
 0.0625     0.125      0.25       0.5        1.        ]


In [12]:
ranges = []
clip_values = []
idx = 0

while True:
    ranges.append((scaled_freq_bands[idx], scaled_freq_bands[idx+1]))
    clip_values.append(idx+1)
    idx += 1
    
    if idx == len(scaled_freq_bands)-1:
        break

In [13]:
ranges

[(0.0, 0.001953125),
 (0.001953125, 0.00390625),
 (0.00390625, 0.0078125),
 (0.0078125, 0.015625),
 (0.015625, 0.03125),
 (0.03125, 0.0625),
 (0.0625, 0.125),
 (0.125, 0.25),
 (0.25, 0.5),
 (0.5, 1.0)]

In [288]:
# clip scaled_dist_map
clip_dist_map = np.zeros_like(dists)

for i, (lower, upper) in enumerate(ranges):
    clip_dist_map[(dists>=lower) & (dists < upper)] = clip_values[i]

In [295]:
dist_thresholds = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

for i in range(21):
    print(dist_thresholds[i // 2])

TypeError: 'tuple' object is not callable

In [14]:
list(range(1, 10+1))[::-1]

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]